Referencias:
SNMP
* http://www.haindl.org/doc/tcpipv1/snmp_sim.htm
* http://www.tcpipguide.com/free/t_SNMPVersion1SNMPv1MessageFormat-3.htm
* https://support.huawei.com/enterprise/br/doc/EDOC1100086963
* https://www.incibe-cert.es/en/blog/snmp-it-simple-name-implies
* https://blog.domotz.com/own-the-networks/snmp-port-number/
Struct:
https://docs.python.org/3/library/struct.html

In [335]:
from struct import pack, unpack
import socket

## Montando o datagram

![structure](assets/snmp_structure.png)  

![data](assets/snmp_data.png)

### Montando VarbindList

* System Description: .1.3.6.1.2.1.1.1.0
* System Uptime: .1.3.6.1.2.1.1.3.0
* System Name: .1.3.6.1.2.1.1.5.0

In [426]:
dados = b'mensagem a ser passada por snmp'


In [429]:
snmpValue = pack("bb{}s".format(len(dados)),4,len(dados),dados) #byte 0x05 indica que dataType do que to enviando é nulo

In [459]:
OID_CODE = '1.2.1.1.1.0'.split(".")  #System Description
snmpOid = pack("2b",0x2b, 0x06) + bytearray(int(x) for x in OID_CODE ) # iso.3 + restante do oid

In [460]:
snmpVarbind = snmpOid + snmpValue
snmpVarbind.hex(" ")

'2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [461]:
snmpVarbindSequence = pack('b',6) + len(snmpOid).to_bytes(1,'little') + snmpVarbind 
snmpVarbindSequence.hex(" ")

'06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [462]:
snmpVarbindList = pack('b',0x30) + len(snmpVarbindSequence).to_bytes(1,"little") + snmpVarbindSequence
snmpVarbindList = pack('b',0x30) + len(snmpVarbindList).to_bytes(1,"little") +  snmpVarbindList
snmpVarbindList.hex(" ")

'30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

### Montando PDU

In [463]:
snmpErrorIndex = pack("b",0x0)
snmpPdu = pack("b",0x02) + len(snmpErrorIndex).to_bytes(1,"little") +  snmpErrorIndex + snmpVarbindList
snmpPdu.hex(" ")

'02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [464]:
snmpErrorStatus = pack("b",0x0)
snmpPdu = pack('b',2) + len(snmpErrorStatus).to_bytes(1,"little") + snmpErrorStatus + snmpPdu
snmpPdu.hex(" ")

'02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [465]:
snmpRequestId = pack("b",0x2)
snmpPdu = pack('b',2) + len(snmpRequestId).to_bytes(1,"little") + snmpRequestId + snmpPdu
snmpPdu.hex(" ")

'02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [466]:
snmpPduType =  pack('B',0xa0) # 0xa0 = getRequest, 0xa2 = getResponse
snmpPdu = snmpPduType + len(snmpPdu).to_bytes(1,"little") + snmpPdu
snmpPdu.hex(" ")

'a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

### Montando header

In [467]:
COMMUNITY = b'public'
commLen = len(COMMUNITY)
snmpCommunity = pack("{}s".format(commLen),COMMUNITY)

snmpHeader = pack('b',0x04) + commLen.to_bytes(1,"little") + snmpCommunity + snmpPdu

snmpHeader.hex(" ")

'04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [468]:
snmpVersion = pack('b',0x0)
snmpHeader = pack('b',0x2) + len(snmpVersion).to_bytes(1,"little") + snmpVersion + snmpHeader

snmpHeader.hex(" ")

'02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [469]:
snmpMessage = pack('b',0x30) + len(snmpHeader).to_bytes(1,"little") + snmpHeader
snmpMessage.hex(" ")

'30 45 02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70'

In [470]:
print('Mensagem SNMP em bytes:\n{}\n\nMensagem SNMP em Hex:\n{}'.format(snmpMessage, snmpMessage.hex(" ")))

Mensagem SNMP em bytes:
b'0E\x02\x01\x00\x04\x06public\xa08\x02\x01\x02\x02\x01\x00\x02\x01\x000-0+\x06\x08+\x06\x01\x02\x01\x01\x01\x00\x04\x1fmensagem a ser passada por snmp'

Mensagem SNMP em Hex:
30 45 02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 01 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70


## Montando o socket

In [471]:
ipDest = b'127.0.0.1' #b"187.41.236.33"
portDest = 161
s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

In [472]:
s.sendto(snmpMessage,(ipDest,portDest))
try:
	Rxbuf = s.recv(2000)
	print ('Resposta Recebida!')
except socket.timeout:
	print ('time out!!!')
s.close()

Resposta Recebida!


In [490]:
try:
    print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8")))
except UnicodeDecodeError:
    print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}\n\nOBS.: a string pode conter erros de decodificação".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8",errors='ignore')))

Bytes:
b'0l\x02\x01\x00\x04\x06public\xa2_\x02\x01\x02\x02\x01\x00\x02\x01\x000T0R\x06\x08+\x06\x01\x02\x01\x01\x01\x00\x04FLinux tom 5.10.0-13-amd64 #1 SMP Debian 5.10.106-1 (2022-03-17) x86_64'

Hex:
30 6c 02 01 00 04 06 70 75 62 6c 69 63 a2 5f 02 01 02 02 01 00 02 01 00 30 54 30 52 06 08 2b 06 01 02 01 01 01 00 04 46 4c 69 6e 75 78 20 74 6f 6d 20 35 2e 31 30 2e 30 2d 31 33 2d 61 6d 64 36 34 20 23 31 20 53 4d 50 20 44 65 62 69 61 6e 20 35 2e 31 30 2e 31 30 36 2d 31 20 28 32 30 32 32 2d 30 33 2d 31 37 29 20 78 38 36 5f 36 34

String:
0l public_  0T0R+ FLinux tom 5.10.0-13-amd64 #1 SMP Debian 5.10.106-1 (2022-03-17) x86_64

OBS.: a string pode conter erros de decodificação


In [491]:
def programaResumido(OID_CODE):
	dados = b'mensagem a ser passada por snmp'
	snmpValue = pack("bb{}s".format(len(dados)),4,len(dados),dados) #byte 0x05 indica que dataType do que to enviando é nulo
	OID_CODE = '1.2.1.1.3.0'.split(".")
	snmpOid = pack("2b",0x2b, 0x06) + bytearray(int(x) for x in OID_CODE ) # iso.3 + restante do oid
	snmpVarbind = snmpOid + snmpValue
	snmpVarbindSequence = pack('b',6) + len(snmpOid).to_bytes(1,'little') + snmpVarbind 
	snmpVarbindList = pack('b',0x30) + len(snmpVarbindSequence).to_bytes(1,"little") + snmpVarbindSequence
	snmpVarbindList = pack('b',0x30) + len(snmpVarbindList).to_bytes(1,"little") +  snmpVarbindList
	snmpErrorIndex = pack("b",0x0)
	snmpPdu = pack("b",0x02) + len(snmpErrorIndex).to_bytes(1,"little") +  snmpErrorIndex + snmpVarbindList
	snmpErrorStatus = pack("b",0x0)
	snmpPdu = pack('b',2) + len(snmpErrorStatus).to_bytes(1,"little") + snmpErrorStatus + snmpPdu
	snmpRequestId = pack("b",0x2)
	snmpPdu = pack('b',2) + len(snmpRequestId).to_bytes(1,"little") + snmpRequestId + snmpPdu
	snmpPduType =  pack('B',0xa0) # 0xa0 = getRequest, 0xa2 = getResponse
	snmpPdu = snmpPduType + len(snmpPdu).to_bytes(1,"little") + snmpPdu
	COMMUNITY = b'public'
	commLen = len(COMMUNITY)
	snmpCommunity = pack("{}s".format(commLen),COMMUNITY)
	snmpHeader = pack('b',0x04) + commLen.to_bytes(1,"little") + snmpCommunity + snmpPdu
	snmpVersion = pack('b',0x0)
	snmpHeader = pack('b',0x2) + len(snmpVersion).to_bytes(1,"little") + snmpVersion + snmpHeader
	snmpMessage = pack('b',0x30) + len(snmpHeader).to_bytes(1,"little") + snmpHeader
	print('Mensagem SNMP em bytes:\n{}\n\nMensagem SNMP em Hex:\n{}\n\n'.format(snmpMessage, snmpMessage.hex(" ")))
	ipDest = b'127.0.0.1' #b"187.41.236.33"
	portDest = 161
	s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
	s.sendto(snmpMessage,(ipDest,portDest))
	try:
		Rxbuf = s.recv(2000)
		print ('Resposta Recebida!')
	except socket.timeout:
		print ('time out!!!')
	s.close()
	try:
		print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8")))
	except UnicodeDecodeError:
		print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}\n\nOBS.: a string pode conter erros de decodificação".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8",errors='ignore')))

Mensagem SNMP em bytes:
b'0E\x02\x01\x00\x04\x06public\xa08\x02\x01\x02\x02\x01\x00\x02\x01\x000-0+\x06\x08+\x06\x01\x02\x01\x01\x03\x00\x04\x1fmensagem a ser passada por snmp'

Mensagem SNMP em Hex:
30 45 02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 03 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70


Resposta Recebida!
Bytes:
b'0)\x02\x01\x00\x04\x06public\xa2\x1c\x02\x01\x02\x02\x01\x00\x02\x01\x000\x110\x0f\x06\x08+\x06\x01\x02\x01\x01\x03\x00C\x03\x05d\xb8'

Hex:
30 29 02 01 00 04 06 70 75 62 6c 69 63 a2 1c 02 01 02 02 01 00 02 01 00 30 11 30 0f 06 08 2b 06 01 02 01 01 03 00 43 03 05 64 b8

String:
0) public  00+ Cd

OBS.: a string pode conter erros de decodificação


### Resumindo o programa e modificando o OID

In [493]:
def programaResumido(OID_CODE):
    dados = b'mensagem a ser passada por snmp'
    snmpValue = pack("bb{}s".format(len(dados)),4,len(dados),dados) #byte 0x05 indica que dataType do que to enviando é nulo
    OID_CODE = OID_CODE.split(".")
    snmpOid = pack("2b",0x2b, 0x06) + bytearray(int(x) for x in OID_CODE ) # iso.3 + restante do oid
    snmpVarbind = snmpOid + snmpValue
    snmpVarbindSequence = pack('b',6) + len(snmpOid).to_bytes(1,'little') + snmpVarbind 
    snmpVarbindList = pack('b',0x30) + len(snmpVarbindSequence).to_bytes(1,"little") + snmpVarbindSequence
    snmpVarbindList = pack('b',0x30) + len(snmpVarbindList).to_bytes(1,"little") +  snmpVarbindList
    snmpErrorIndex = pack("b",0x0)
    snmpPdu = pack("b",0x02) + len(snmpErrorIndex).to_bytes(1,"little") +  snmpErrorIndex + snmpVarbindList
    snmpErrorStatus = pack("b",0x0)
    snmpPdu = pack('b',2) + len(snmpErrorStatus).to_bytes(1,"little") + snmpErrorStatus + snmpPdu
    snmpRequestId = pack("b",0x2)
    snmpPdu = pack('b',2) + len(snmpRequestId).to_bytes(1,"little") + snmpRequestId + snmpPdu
    snmpPduType =  pack('B',0xa0) # 0xa0 = getRequest, 0xa2 = getResponse
    snmpPdu = snmpPduType + len(snmpPdu).to_bytes(1,"little") + snmpPdu
    COMMUNITY = b'public'
    commLen = len(COMMUNITY)
    snmpCommunity = pack("{}s".format(commLen),COMMUNITY)
    snmpHeader = pack('b',0x04) + commLen.to_bytes(1,"little") + snmpCommunity + snmpPdu
    snmpVersion = pack('b',0x0)
    snmpHeader = pack('b',0x2) + len(snmpVersion).to_bytes(1,"little") + snmpVersion + snmpHeader
    snmpMessage = pack('b',0x30) + len(snmpHeader).to_bytes(1,"little") + snmpHeader
    print('Mensagem SNMP em bytes:\n{}\n\nMensagem SNMP em Hex:\n{}\n\n'.format(snmpMessage, snmpMessage.hex(" ")))
    ipDest = b'127.0.0.1' #b"187.41.236.33"
    portDest = 161
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.sendto(snmpMessage,(ipDest,portDest))
    try:
        Rxbuf = s.recv(2000)
        print ('Resposta Recebida!')
    except socket.timeout:
        print ('time out!!!')
    s.close()
    try:
        print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8")))
    except UnicodeDecodeError:
        print("Bytes:\n{}\n\nHex:\n{}\n\nString:\n{}\n\nOBS.: a string pode conter erros de decodificação".format(Rxbuf,Rxbuf.hex(" "),Rxbuf.decode("utf-8",errors='ignore')))

In [495]:
programaResumido("1.2.1.1.3.0")

Mensagem SNMP em bytes:
b'0E\x02\x01\x00\x04\x06public\xa08\x02\x01\x02\x02\x01\x00\x02\x01\x000-0+\x06\x08+\x06\x01\x02\x01\x01\x03\x00\x04\x1fmensagem a ser passada por snmp'

Mensagem SNMP em Hex:
30 45 02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 03 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70


Resposta Recebida!
Bytes:
b'0)\x02\x01\x00\x04\x06public\xa2\x1c\x02\x01\x02\x02\x01\x00\x02\x01\x000\x110\x0f\x06\x08+\x06\x01\x02\x01\x01\x03\x00C\x03\x05\xc7\xcf'

Hex:
30 29 02 01 00 04 06 70 75 62 6c 69 63 a2 1c 02 01 02 02 01 00 02 01 00 30 11 30 0f 06 08 2b 06 01 02 01 01 03 00 43 03 05 c7 cf

String:
0) public  00+ C

OBS.: a string pode conter erros de decodificação


In [496]:
programaResumido("1.2.1.1.5.0")

Mensagem SNMP em bytes:
b'0E\x02\x01\x00\x04\x06public\xa08\x02\x01\x02\x02\x01\x00\x02\x01\x000-0+\x06\x08+\x06\x01\x02\x01\x01\x05\x00\x04\x1fmensagem a ser passada por snmp'

Mensagem SNMP em Hex:
30 45 02 01 00 04 06 70 75 62 6c 69 63 a0 38 02 01 02 02 01 00 02 01 00 30 2d 30 2b 06 08 2b 06 01 02 01 01 05 00 04 1f 6d 65 6e 73 61 67 65 6d 20 61 20 73 65 72 20 70 61 73 73 61 64 61 20 70 6f 72 20 73 6e 6d 70


Resposta Recebida!
Bytes:
b'0)\x02\x01\x00\x04\x06public\xa2\x1c\x02\x01\x02\x02\x01\x00\x02\x01\x000\x110\x0f\x06\x08+\x06\x01\x02\x01\x01\x05\x00\x04\x03tom'

Hex:
30 29 02 01 00 04 06 70 75 62 6c 69 63 a2 1c 02 01 02 02 01 00 02 01 00 30 11 30 0f 06 08 2b 06 01 02 01 01 05 00 04 03 74 6f 6d

String:
0) public  00+ tom

OBS.: a string pode conter erros de decodificação
